Assign Geo-Location to Your Camera

In [5]:
import pandas as pd

df = pd.read_csv("/content/traffic_counts_per_minute.csv")

CAMERA_LAT = 12.9716   # Example: Bengaluru
CAMERA_LON = 77.5946

df["latitude"] = CAMERA_LAT
df["longitude"] = CAMERA_LON

df

,minute,vehicle,count,latitude,longitude
0,0,car,66,12.9716,77.5946
1,0,truck,8,12.9716,77.5946


Install dependencies

In [6]:
!pip install folium

Heatmap Generation Code

In [7]:
import folium
from folium.plugins import HeatMap

m = folium.Map(
    location=[CAMERA_LAT,CAMERA_LON],
    zoom_start=16,
)

heat_data = [
    [row["latitude"], row["longitude"], row["count"]]
    for _, row in df.iterrows()
]

HeatMap(
    heat_data,
    radius=30,
    blur=20,
    max_zoom=17,
).add_to(m)

m

Creating the data

In [8]:
import pandas as pd
import io

csv_data = io.StringIO("""minute,vehicle,count,latitude,longitude
0,car,66,12.9716,77.5946
0,truck,8,12.9716,77.5946
1,car,52,12.9716,77.5946
1,truck,6,12.9716,77.5946""")

df_new = pd.read_csv(csv_data)
df_new.to_csv('traffic_counts_per_minute.csv', index=False)

print('CSV file "traffic_counts_per_minute.csv" created successfully.')

CSV file "traffic_counts_per_minute.csv" created successfully.


Prepare time-based heatmap data

In [9]:
import pandas as pd

# Load your CSV
df = pd.read_csv("traffic_counts_per_minute.csv")

# Aggregate total vehicles per minute per location
heat_df = (
    df.groupby(["minute", "latitude", "longitude"])["count"]
    .sum()
    .reset_index()
)

heat_df.head()


,minute,latitude,longitude,count
0,0,12.9716,77.5946,74
1,1,12.9716,77.5946,58


In [11]:
time_indexed_data = []

for minute in sorted(heat_df["minute"].unique()):
    frame = heat_df[heat_df["minute"] == minute]
    points = frame[["latitude", "longitude", "count"]].values.tolist()
    time_indexed_data.append(points)

len(time_indexed_data)  # number of time steps



2

In [12]:
import folium
from folium.plugins import HeatMapWithTime

# Center of the map (camera location)
CENTER_LAT = heat_df["latitude"].iloc[0]
CENTER_LON = heat_df["longitude"].iloc[0]

m = folium.Map(
    location=[CENTER_LAT, CENTER_LON],
    zoom_start=17,
    tiles="cartodbpositron"
)

HeatMapWithTime(
    time_indexed_data,
    radius=25,
    auto_play=True,
    max_opacity=0.8,
    gradient={0.2: "blue", 0.4: "lime", 0.6: "orange", 0.9: "red"}
).add_to(m)

m
